In [1]:
from termcolor import cprint

def print_board(board, bolded=None):
    """A debugging function to print you board in a pretty way"""
    n = len(board)
    # For every row but the last
    for row_idx, row in enumerate(board[:-1]):
        # Print the row as a string with a line below
        if bolded and row_idx==bolded[0]:
            cprint("|".join(row[:bolded[1]]), None, attrs=["underline"], end='|' if bolded[1] != 0 else '')
            cprint(row[bolded[1]], None, attrs=["underline", "bold"], end='')
            if bolded[1] != len(row) - 1:
                cprint("|" + "|".join(row[bolded[1]+1:]), None, attrs=["underline"], end='')
            print()
        else:
            cprint("|".join(row), None, attrs=["underline"])
    row = board[-1]
    if bolded and bolded[0] == len(board) - 1:
        print("|".join(row[:bolded[1]]), end='|' if bolded[1] != 0 else '')
        cprint(row[bolded[1]], None, attrs=["bold"], end='')
        if bolded[1] != len(row) - 1:
            cprint("|" + "|".join(row[bolded[1]+1:]), None, end='')
        print()
    else:
        print("|".join(row))


def other_stone(stone):
    return "X" if stone == "O" else "O"

def consecutive_k(row, k, stone):
    desired_row = [stone] * k
    return sum(desired_row == row[i:i+k] for i in range(len(row) - k + 1))

def get_downdiag(board, row_idx, col_idx):
    return [board[row_idx + i][col_idx + i] for i in range(min(len(board) - row_idx, len(board) - col_idx))]

def get_updiag(board, row_idx, col_idx):
    return [board[row_idx - i][col_idx + i] for i in range(min(row_idx + 1, len(board) - col_idx))]

def winner_stone(board, stone):
    k_count = 0
    k = 5
    for row in board:
        row = list(row)
        k_count += consecutive_k(row, k, stone)

    for col_idx in range(len(board)):
        bl = [row[col_idx] for row in board]
        k_count += consecutive_k(bl, k, stone)

    for row_idx in range(len(board)):
        bl = get_updiag(board, row_idx, 0)
        k_count += consecutive_k(bl, k, stone)
        bl = get_downdiag(board, row_idx, 0)
        k_count += consecutive_k(bl, k, stone)

    for col_idx in range(len(board[0])):
        bl = get_updiag(board, len(board) - 1, col_idx)
        k_count += consecutive_k(bl, k, stone)
        bl = get_downdiag(board, 0, col_idx)
        k_count += consecutive_k(bl, k, stone)
    
    return k_count > 0

def complete_board(board):
    return winner_stone(board, "X") or winner_stone(board, "O") or not any('-' in row for row in board)


In [2]:
import random
import heapq
import numpy as np
from tqdm.notebook import tqdm
import copy

class Strategy:
    def __init__(self, stone, eval_function, max_depth=1):
        self.stone = stone
        self.opponent_stone = other_stone(stone)
        self.max_depth = max_depth
        self.nodes = 0
        self.pruned = 0
        self.eval_function=eval_function 
    
    def alphabeta_search(self, board, depth, stone, alpha, beta):
        if depth == 0 or complete_board(board):
            score = self.eval_function(board, stone)
            score = -score if stone == self.opponent_stone else score
            self.nodes += 1
            return score, (None, None)
        
        row_arr, col_arr = np.where(board == '-')
        open_spaces = list(zip(row_arr, col_arr))
        if stone == self.stone:
            best_score = -np.infty
            best_move = (None, None)
            lop = len(open_spaces)
            c = 0
            for row, col in open_spaces:
                new_board = copy.deepcopy(board)
                new_board[row][col] = stone
                score, move = self.alphabeta_search(new_board, depth-1, other_stone(stone), alpha, beta)
                best_score = max(score, best_score)
                if best_score >= beta:
                    self.pruned += (lop - c)**depth
                    break
                elif best_score > alpha:
                    best_move = (row, col)
                    alpha = best_score
        else:
            best_score = np.infty
            best_move = (None, None)
            lop = len(open_spaces)
            c = 0
            for row, col in open_spaces:
                c += 1
                new_board = copy.deepcopy(board)
                new_board[row][col] = stone
                score, move = self.alphabeta_search(new_board, depth-1, other_stone(stone), alpha, beta)
                best_score = min(score, best_score)
                if best_score <= alpha:
                    self.pruned += (lop - c)**depth
                    break
                elif best_score < beta:
                    best_move = (row, col)
                    beta = best_score
                
        self.nodes += 1
        return best_score, best_move
        
                    
    def get_move(self, board, max_nodes = 100):
        score, move = self.alphabeta_search(board, self.max_depth, self.stone, -np.inf, np.inf)
        return move
            


In [3]:
def random_eval(_1, _2):
    return random.random()

def student_eval(board, stone):
  points = 0
  if stone == "X": #Checking which stone the player is
    opponent = "O" #Saying what stone opponent is
  elif stone == "O":  #Checking which stone the player is
    opponent = "X" #Saying what stone opponent is NOTE: I(Ulises) changed the == to = in this line
  k = len(board) #Establishing the n in the nXn board
  rowlist = []
  for r in range(k):
    for c in range(k):
      rowlist.append(board[r][c])
    if consecutive_five(rowlist,stone): # If you won then get 1000 points
      points = points + 1000
      break
    elif consecutive_five(rowlist,opponent): # If opponent won lose 1000 points
      points = points + 1000
      break
    collist = get_column(board, r) # Get list of column at the index of r
    if consecutive_five(collist,stone): # If you won then get 1000 points
      points = points + 1000
    elif  consecutive_five(collist,opponent): # If opponent won then get 1000 points
      points = points - 1000
  for r in range(k):
    for c in range(k-1,-1,-1):
      updiag = get_updiag(board,r,c)
      if consecutive_five(updiag,stone):
        points = points + 1000
      elif consecutive_five(updiag,opponent):
        points = points - 1000
  for r in range(k):
   for c in range(k):
     downdiag = get_downdiag(board,k-1,k-1)
     if consecutive_five(downdiag,stone):
       points = points + 1000
     elif consecutive_five(downdiag,opponent):
       points = points - 1000
  rowlist = []
  for r in range(k):
    for c in range(k):
      rowlist.append(board[r][c])
    if connect_four(rowlist,stone): # If you won then get 1000 points
      points = points + 1000
      break
    elif connect_four(rowlist,opponent): # If opponent won lose 1000 points
      points = points + 1000
      break
    collist = get_column(board, r) # Get list of column at the index of r
    if connect_four(collist,stone): # If you won then get 1000 points
      points = points + 1000
    elif  connect_four(collist,opponent): # If opponent won then get 1000 points
      points = points - 1000
  for r in range(k):
    for c in range(k-1,-1,-1):
      updiag = get_updiag(board,r,c)
      if connect_four(updiag,stone):
        points = points + 1000
      elif connect_four(updiag,opponent):
        points = points - 1000
  for r in range(k):
   for c in range(k):
     downdiag = get_downdiag(board,k-1,k-1)
     if connect_four(downdiag,stone):
       points = points + 1000
     elif connect_four(downdiag,opponent):
       points = points - 1000
  if points != 1000: # 1000 points mean that there is a winner and a loser so no need to check the board anymore
      for r in range(k):
        for c in range(k):
          rowlist.append(board[r][c])
        points = points + rowpoints(rowlist,stone,opponent)
        collist = get_column(board, r)
        points = points + rowpoints(collist,stone,opponent)
      for r in range(k):
        for c in range(k-1,-1,-1):
          updiag = get_updiag(board,r,c)    
          points = points + rowpoints(updiag,stone,opponent)
      for r in range(k):
        for c in range(k):
          downdiag = get_downdiag(board,k-1,k-1)
        points = points + rowpoints(downdiag,stone,opponent)    
  return points # Replace this!

In [11]:
def play_game(board, strategy_1, strategy_2):
    player1_turn = True
    turn_count = 0
    n = len(board)
    m = len(board[0])
    while not complete_board(board) and turn_count < m*n:
        if player1_turn:
            row, col = strategy_1.get_move(board)
            board[row][col] = "X"
            if winner_stone(board, "X"):
                print("Student won!")
        else:
            row, col = strategy_2.get_move(board)
            board[row][col] = "O"
            if winner_stone(board, "O"):
                print("Computer won!")
        turn_count += 1
        print(turn_count, "X" if player1_turn else "O", f"{row},{col}")
        print_board(board, (row, col))
        board[row][col] = "X" if player1_turn else "O"
        player1_turn = not player1_turn
    if not winner_stone(board, "O") and  not winner_stone(board, "X"):
        print("Tie!")
    return complete_board(board)



big_board = np.array([list(x) for x in """
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-|-|-|-|-|-|-|-""".replace("|", "").split('\n')[1:]])

small_board = np.array([list(x) for x in """
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-""".replace("|", "").split('\n')[1:]])

p1 = Strategy("X", student_eval, max_depth=1) # Do not change the search depths!
p2 = Strategy("O", random_eval, max_depth=1)
play_game(small_board, p1, p2) # Start with the small board, it is better for debugging.

1 X 0,0
X|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
2 O 2,2
X|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|O|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
3 X 1,3
X|-|-|-|-|-|-|-
-|-|-|X|-|-|-|-
-|-|O|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
4 O 1,1
X|-|-|-|-|-|-|-
-|O|-|X|-|-|-|-
-|-|O|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
5 X 3,1
X|-|-|-|-|-|-|-
-|O|-|X|-|-|-|-
-|-|O|-|-|-|-|-
-|X|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
6 O 2,1
X|-|-|-|-|-|-|-
-|O|-|X|-|-|-|-
-|O|O|-|-|-|-|-
-|X|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
7 X 2,3
X|-|-|-|-|-|-|-
-|O|-|X|-|-|-|-
-|O|O|X|-|-|-|-
-|X|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
-|-|-|-|-|-|-|-
8 O 7,1
X|-|-|-|-|-|-|-
-|O|-|X|-|-|-|-
-|O|O|X|

True

In [5]:
def get_downdiag(ttt_board, row_idx, col_idx): # Can be top and left sides
    dialist = []
    r = row_idx
    c = col_idx
    while r > -1 and c > -1:
      dialist.append(ttt_board[r][c])
      r -= 1
      c -= 1
    return dialist

In [6]:
def get_updiag(ttt_board, row_idx, col_idx): #Can be bottom and left sides
    dialist = []
    r = row_idx
    c = col_idx
    while r > -1 and c < len(ttt_board):
      dialist.append(ttt_board[r][c])
      r -= 1
      c += 1
    return dialist

In [7]:
def get_column(ttt_board, col_idx):
  coll = len(ttt_board)
  collist = []
  for c in range(0,coll):
    collist.append(ttt_board[c][col_idx])
  return collist

In [8]:
def consecutive_five(input_list, stone):
    for i in range(len(input_list)):
      check = 0
      if input_list[i] == stone:
        check = 0
        o = i
        while o < len(input_list) and input_list[o] == stone:
          check = check + 1
          o += 1
      if check >= 5:
        return True
    return False

In [9]:
def connect_four(input_list, stone):
    for i in range(len(input_list)):
      check = 1
      if input_list[i] == stone:
        check = 0
        o = i
        while o < len(input_list) and input_list[o] == stone:
          check = check + 1
          o += 1
      if check >= 4:
        return True
    return False

In [10]:
def rowpoints(input_list, stone,opponent):  #This will return how many consecutive stones there are for the stone given
    points = 0
    for i in range(len(input_list)):
      if input_list[i] == stone:
          if i-1 > -1 and input_list[i-1] == "-":
            points = points + 10
          elif i+1 < len(input_list) and (input_list[i+1] == "-" or input_list[i+1] == stone):
            points = points + 10
          elif i-1 > -1 and i+1 < len(input_list) and input_list[i-1] == opponent and input_list[i+1] == opponent:
            points = points - points
    return points